In [1]:
try:
    from google.colab import drive
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

if COLAB:
  !sudo apt-get install -y xvfb ffmpeg
  !pip install -q 'gym==0.10.11'
  !pip install -q 'imageio==2.4.0'
  !pip install -q PILLOW
  !pip install -q 'pyglet==1.3.2'
  !pip install -q pyvirtualdisplay
  !pip install -q --upgrade tensorflow-probability
  !pip install -q tf-agents

Note: using Google CoLab
Reading package lists... Done
Building dependency tree       
Reading state information... Done
ffmpeg is already the newest version (7:3.4.8-0ubuntu0.2).
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 784 kB of archives.
After this operation, 2,270 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.9 [784 kB]
Fetched 784 kB in 1s (1,277 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting

In [2]:
! wget http://www.atarimania.com/roms/Roms.rar
! mkdir /content/ROM/
! unrar e /content/Roms.rar /content/ROM/
! python -m atari_py.import_roms /content/ROM/

--2021-07-14 15:51:01--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar’

Roms.rar            100%[===================>]  10.61M   819KB/s    in 14s     

2021-07-14 15:51:15 (791 KB/s) - ‘Roms.rar’ saved [11128004/11128004]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from /content/Roms.rar

Extracting  /content/ROM/HC ROMS.zip                                      36%  OK 
Extracting  /content/ROM/ROMS.zip                                         74% 99%  OK 
All OK
copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (P

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
import tensorflow as tf
import numpy as np
import gym
import scipy.misc
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet_v2 import preprocess_input, decode_predictions
from tensorflow.keras import layers
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import random_uniform, glorot_uniform, constant, identity
from tensorflow.python.framework.ops import EagerTensor
import matplotlib.pyplot as plt
from collections import deque
from skimage import transform
import os

In [5]:
env = gym.make('MsPacman-v0')
observation = env.reset()
input_shape = observation.shape
classes = env.action_space.n
# print(input_shape, classes)

In [6]:
def identity_block(X, f, filters, training=True, initializer=random_uniform):

    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    
    ## Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X_shortcut,X])
    X = Activation('relu')(X)

    return X

In [7]:
def convolutional_block(X, f, filters, s = 2, training=True, initializer=glorot_uniform):

    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X

    ##### MAIN PATH #####
    
    # First component of main path glorot_uniform(seed=0)
    X = Conv2D(filters = F1, kernel_size = 1, strides = (s, s), padding='valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training=training)
    X = Activation('relu')(X)

    ## Second component of main path (≈3 lines)
    X = Conv2D(filters = F2, kernel_size = f, strides = (1,1), padding = 'same', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    X = Activation('relu')(X)

    ## Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = 1, strides = (1,1), padding = 'valid', kernel_initializer = initializer(seed=0))(X)
    X = BatchNormalization(axis = 3)(X, training = training) # Default axis
    
    ##### SHORTCUT PATH ##### (≈2 lines)
    X_shortcut = Conv2D(filters = F3, kernel_size = 1, strides = (s,s), padding = 'valid', kernel_initializer = initializer(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3)(X_shortcut, training = training) # Default axis
    
    # Final step: 
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)
    
    return X

In [8]:
def ResNet50(input_shape, classes): # input and classes depends on gym environment
    """
    Stage-wise implementation of the architecture of the popular ResNet50:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> FLATTEN -> DENSE 

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], s = 1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])
    
    ## Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128, 128, 512], s = 2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512]) 
    
    ## Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256, 256, 1024], s = 2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    ## Stage 5 (≈3 lines)
    X = convolutional_block(X, f = 3, filters = [512, 512, 2048], s = 2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    ## AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D(pool_size = (2,2))(X)

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X)

    return model

In [9]:
def Deepminds_model(input_shape, classes):
        
        
    X_input = Input(input_shape)
    
    X = Conv2D(32, (8, 8), strides = (4, 4), kernel_initializer = glorot_uniform(seed=0))(X_input)
    X = Activation('relu')(X)
    X = Conv2D(64, (4, 4), strides = (2, 2), kernel_initializer = glorot_uniform(seed=0))(X)
    X = Activation('relu')(X)
    X = Conv2D(64, (3, 3), strides = (1, 1), kernel_initializer = glorot_uniform(seed=0))(X)
    X = Activation('relu')(X)
    X = Flatten()(X)
    X = Dense(512, activation='relu', kernel_initializer = glorot_uniform(seed=0))(X)
    X = Dense(classes, activation='softmax', kernel_initializer = glorot_uniform(seed=0))(X)
    
    model = Model(inputs = X_input, outputs = X)
    
    

In [10]:
def scale_lumininance(img):
    return np.dot(img[...,:3],[0.299,0.587,0.114])

In [11]:
def preprocess_observation(obs):
    """
    To reduce the computation load we preprocess the date obtained from the atari-game env by
    reducing the resolution and adjusting the color.
    
    DeepMind took the maximum pixel value over subsequent frames to reduce flickering
    caused by the limitations of the Atari platform and then scale it from its current 
    210×160×3 resolution to 84×84.
    
    To convert this, we will take the luminance channel (denoted as Y) from the image, which is the our RGB channel, and apply linear weights to 
    the channel to transform it according to the relative luminance.
    Y = 0.299R + 0.587G + 0.114B
    """
    
    obs_gray = scale_lumininance(obs)
    obs_trans = transform.resize(obs_gray,(84,84))
    return np.moveaxis(obs_trans, 1, 0)

In [12]:
def initialization(input_shape , classes, buffer_size = 50000):
    """
    The target_network and prediction_network are initialized as ResNet50/DeepMind's CNN architecture with trainability
    of prediction_network set to False.
    
    The Experience Reply is initialized as a deque whose each element 
    is a tuple (current state, action, reward , done, next state)
    Few actions are executed with the environment to bootstrap the replay data.
    """
    global target_network,prediction_network,experience_replay
#     target_network = ResNet50(input_shape,classes)
    target_network = Deepminds_model(input_shape,classes)  
    # print(target_network.summary())
    
#     prediction_network = ResNet50(input_shape,classes)
    prediction_network = Deepminds_model(input_shape,classes)
    # print(prediction_network.summary())
    
    experience_replay = deque(maxlen = buffer_size)
    
    observation = env.reset()
    for i in range(buffer_size):
        
        current_state = preprocess_observation(observation)
        action = env.action_space.sample()
        observation, reward, done , _ = env.step(action)
        next_state = preprocess_observation(observation)
        
        sample_observation = (current_state,action,reward,done,next_state)
        experience_replay.append(sample_observation)

In [13]:
def plot(rewards , episode, save_fig = True):
    plt.plot(rewards,[i+1 for i in range(episode)], linewidth=4, label = "DDQN")
    plt.xlabel("Episode")
    plt.ylabel("episode_reward")
    leg = plt.legend(loc='upper left', shadow=True)
    plt.grid()
    ax = plt.gca()
    plt.xlim([1, episode])
    
    output_dir = os.path.join(os.getcwd(), "output")
    if save_fig:
        if not os.path.exists(output_dir): os.mkdir(output_dir)
        plt.savefig(os.path.join(output_dir, "plot-{}th-episode".format(episode)), bbox_inches="tight")
    else:
        plt.show()
    plt.close()

In [14]:
def eps_greedy_action(Q,eps):
    if np.random.random() < eps: # explore
        return np.random.randint(env.action_space.n)
    else: # exploit
        return np.random.choice(np.flatnonzero(Q == Q.max()))

In [15]:
def experiment(N_episodes,gamma,learning_rate, c, batch_size,eps):
    
    optimizer = tf.keras.optimizers.RMSprop(learning_rate) # used by deepminds
    loss_fn = tf.keras.losses.MeanSquaredError()
    
    for episode in range(N_episodes):
        time_step = 0
        observation = env.reset()
        rewards = []
        while True:
            env.render()
            with tf.GradientTape() as tape:
                current_state = preprocess_observation(observation) 
                Q = prediction_network(current_state, training=False)
                action = eps_greedy_action(Q,eps) 
                new_observation , reward , done , _ = env.step(action)
                rewards.append(reward)
                next_state = preprocess_observation(new_observation)
                sample_observation = (current_state, action, reward, done, next_state)
                experience_replay.append(sample_observation)
                minibatch = random.sample(list(experience_replay),batch_size)
                target = []
                for (current_state, action, reward, done, next_state) in minibatch:
                    if done:
                        target.append(reward)
                    else:
                        Q_target = target_network(next_state,training = False)
                        target.append(reward + gamma * Q.max())
                loss_value = loss_fn(target , Q)
                grads = tape.gradient(loss_value, prediction_network.trainable_weights)
                optimizer.apply_gradients(zip(grads, prediction_network.trainable_weights))
                observation = new_observation
                if (episode%c)==0: 
                    target_network.trainable_weights = prediction_network.trainable_weights
                    prediction_network.save("/output/models/model-{}.h5".format(episode/c))
                
                if done:
                    print("Episode {} finished after {} timesteps".format(i_episode+1, time_step+1))
                    plot(rewards,episode)
                    break
 

In [ ]:
learning_rate = 0.0075
gamma = 0.8
c = 100 # not sure
eps = 0.5
batch_size = 64
N_episodes = 10000

initialization(input_shape,classes)
experiment(N_episodes,gamma,learning_rate, c, batch_size,eps)
env.close()

/home/utkarsh
